In [ ]:
#This code will find the filler word from audio
#Inputs: audio files with .wav format
#Output: time when filler word occurs
#Files this notebook refers: Excel sheet 'https://docs.google.com/spreadsheets/d/15JyW9TBGHSUhA4c3NOao0V6eYOikZ8x4/edit#gid=1805822620'
#Author: Khushi Pitroda
#Date: 19-July-2021

In [ ]:
#Prerequisite
#.Wav (audio) files
#parselmouth
#data from excel for predefined values

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Change directory 
#Go to shared with me folder create shortcut of shared folder and add it to the my drive so you can access that shared folder from here.

import os 
os.chdir("/content/drive/MyDrive/Paleru_Khushi")    


In [ ]:
pip install praat-parselmouth

     |████████████████████████████████| 10.7 MB 6.9 MB/s 


In [ ]:
import json  
import parselmouth
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Before running this code, make sure the JSON file asrOutput.json is placed in the folder in the drive
#Make sure you have uploaded the audio file in AWS and dowloaded the JSON file in advance.
#Produces the transcript of the audio file

from operator import itemgetter           #for mapping
import math
#provide location of json file of audio here
json_file = "/content/drive/MyDrive/Paleru_Khushi/JSON/asrOutput1.json"

#provide location of audio file here
sound_file = "/content/drive/My Drive/Paleru_Khushi/Filler_words_audios/Person_um1.wav"
  
f = open(json_file)           #create json file by uploading audio files (https://aws.amazon.com/getting-started/hands-on/create-audio-transcript-transcribe/)

data = json.load(f)
#print(data)
#dictonary containing all information 
x = data['results'] ['items']         #fetch result(keyname) from data and from result(sub-dictonary) fetch transcripts stores in list named x  
words_dict = {}
for i in range(1,len(x)):             #for getting each single word, start_time and start_time from the list
  time = []
  start_time = 0                      #initial value
  end_time = 0                        #initial value

  word = x[i]['alternatives'][0]['content']         #get the word- content from dictonary inside the list inside the dicrtonary named alternatives which is inside the list x. print(x) for more understanding.
  confidence = x[i]['alternatives'][0]['confidence'] 
  if word!='.' and word!=',' and word!='?':                       #punctuation doesn't have timestamps so ignoring it
    start_time = x[i]["start_time"]                 #from list getting value of start_time for each word- content
    end_time = x[i]['end_time']                     #from list getting value of end_time for each word- content
    list = [word,start_time,end_time,confidence]
    words_dict[start_time] = list
    #print(word)
    
start_point_aws = 0
end_point_aws = 0
start_point_algo = 0
end_point_algo = 0
#print(words_dict)       
def false_positive(start_point_algo, end_point_algo):
  flag = 1
  global start_point_aws, end_point_aws

  start_point_aws_prev =  start_point_aws
  end_point_aws_prev = end_point_aws
  diffrence_aws_prev = end_point_aws - start_point_aws

  start_point_algo_prev = start_point_algo
  end_point_algo_prev = end_point_algo
  diffrence_algo_prev = end_point_algo - start_point_algo

  diffrence_prev = diffrence_aws_prev - diffrence_algo_prev

  for key in words_dict:
    start_point_aws = float(words_dict[key][1])
    end_point_aws = float(words_dict[key][2])
    confidence = float(words_dict[key][3])
    word_aws = words_dict[key][0]

    diffrence_aws = end_point_aws - start_point_aws
    diffrence_algo = end_point_algo - start_point_algo

    diffrence = diffrence_aws - diffrence_algo
    #applying filters to remove false positives by comparing it with AWS transcript
    if start_point_algo < end_point_aws and end_point_algo < end_point_aws and end_point_algo > start_point_aws:
      #if word is detected from transcript
      if word_aws == "um" or word_aws =="uh": 
        print(f"--------------------------FILLER WORD DETECTED:---------------------\nWord: {word_aws} start_point_aws: {start_point_aws}, end_point_aws: {end_point_aws} Duration : {end_point_aws-start_point_aws}, Actual_duration: {end_point_algo-start_point_algo}, confidence:{confidence}")        #printing all the values
        flag = 0
      elif start_point_aws == start_point_aws_prev:
        #repetition  filter
        if diffrence_algo_prev > diffrence_algo:
          print(f"FALSE POSITIVE WORD DETECTED:\nWord: {word_aws} start_point_aws: {start_point_aws} end_point_aws: {end_point_aws} Duration : {end_point_aws-start_point_aws}, Actual_duration: {end_point_algo-start_point_algo}, confidence:{confidence}")        #printing all the values
          flag = 0
        else:
          print(f"FILLER WORD:\nWord: {word_aws} start_point_aws: {start_point_aws} end_point_aws: {end_point_aws}Duration : {end_point_aws-start_point_aws}, Actual_duration: {end_point_algo-start_point_algo}, confidence:{confidence}")        #printing all the values
        start_point_aws_prev = start_point_aws
        flag = 0
      #if it's false positive
      else:
        print(f"FALSE POSITIVE WORD DETECTED:\nWord: {word_aws} start_point_aws: {start_point_aws} end_point_aws: {end_point_aws} Duration : {end_point_aws-start_point_aws}, Actual_duration: {end_point_algo-start_point_algo}, confidence:{confidence}")        #printing all the values
        start_point_aws_prev = start_point_aws
        end_point_aws_prev = start_point_aws
        start_point_algo_prev = start_point_algo
        end_point_algo_prev = end_point_algo
        flag = 0
      break
    #if nothing is in transcript
    elif start_point_algo < end_point_aws and end_point_algo > start_point_aws:
      print(f"Maybe a filler word: \nstart_point_algo {start_point_algo}, end_point_algo {end_point_algo}, Duration : {end_point_aws-start_point_aws}, confidence:{confidence}")   
      flag = 0
  if flag == 1:
    print("Maybe a filler word:")  
  print(f"start_point_algo {start_point_algo}, end_point_algo {end_point_algo}\n\n")
      



snd = parselmouth.Sound(sound_file) #for importing audio file with filler words
pitch = snd.to_pitch()                 #finding pitch of that filler word

times = pitch.ts()                    #for getting values of different time stamps according to pitch values
hop_dur = .01   #predefined value for time step
num_form = 4    #predefined value for max number of formants = the number of peaks with which the entire spectrum is modelled
max_form_freq = 4500    #predefined value for maximum formant

form = snd.to_formant_burg(time_step=hop_dur, max_number_of_formants=num_form, maximum_formant = max_form_freq, pre_emphasis_from=50.0)   #for getting formant values

form_1_vals = []  #list for storing audio's f1
form_2_vals = []  #list for storing audio's f2
form_3_vals = []  #list for storing audio's f3
form_4_vals = []  #list for storing audio's f4


#values from data in 'https://docs.google.com/spreadsheets/d/15JyW9TBGHSUhA4c3NOao0V6eYOikZ8x4/edit#gid=1805822620'

#for f1
max_formant_f1 = 785       #Average Maximum value from audios (between range of Start_F1 and End_F1) based on excel sheet
min_formant_f1 = 545       #Average Minimum value from audios (between range of Start_F1 and End_F1) based on excel sheet
Max_Consecutive_Positive_Dev_f1 = 60     #Maximum difference between two consecutive formants. (Deviation)
Max_Consecutive_Negative_Dev_f1 = -66    #Minimum difference between two consecutive formants. (Deviation)
min_no_of_continuous_value_f1 = 13
formant_f1 = 0
i=1
arr_f1 = []      #for storing the values of formants 
temp_f1 = []
j=0

#for f2
max_formant_f2 = 2390.201396       #Average Maximum value from audios (between range of Start_F2 and End_F2) based on excel sheet
min_formant_f2 = 1027.659305       #Average Minimum value from audios (between range of Start_F2 and End_F2) based on excel sheet
Max_Consecutive_Positive_Dev_f2 = 116.86   #Maximum difference between two consecutive formants. (Deviation)
Max_Consecutive_Negative_Dev_f2 = -152.86    #Minimum difference between two consecutive formants. (Deviation)
min_no_of_continuous_value_f2 = 13
formant_f2 = 0
arr_f2 = []      #for storing the values of formants 
temp_f2 = []

for dt in times:      #for loop used with average of time instants

    form_1_vals.append(form.get_value_at_time(1,dt))    #storing the values of f2 audio in list
    prev_formant_f1 = formant_f1      #storing formant in prev_formant variable for next iteration
    formant_f1 = form.get_value_at_time(1,dt)    #getting formant value at perticular time
    Dev_f1 = formant_f1-prev_formant_f1    #getting deviation from 2 formant-prev_formant (2 consecutive points)

    
    form_2_vals.append(form.get_value_at_time(2,dt))    #storing the values of f2 audio in list
    prev_formant_f2 = formant_f2      #storing formant in prev_formant variable for next iteration
    formant_f2 = form.get_value_at_time(2,dt)    #getting formant value at perticular time
    Dev_f2 = formant_f2-prev_formant_f2    #getting deviation from 2 formant-prev_formant (2 consecutive points)

    if str(formant_f1) != 'nan' and str(prev_formant_f1) != 'nan'  and str(formant_f2) != 'nan' and str(prev_formant_f2) != 'nan' :    #if both formats are not null
    #checking for conditions based on data, if formant is less than maximum formant acquired from excel, if formant is greater  than minimum formant, if deviation is between minimum and maximum  consecutive deviation then it'll go inside.
      # f1 and f2 filters
      if formant_f1 <max_formant_f1 and formant_f1 > min_formant_f1 and Dev_f1 < Max_Consecutive_Positive_Dev_f1 and Dev_f1>Max_Consecutive_Negative_Dev_f1 and formant_f2 <max_formant_f2 and formant_f2 > min_formant_f2 and Dev_f2<Max_Consecutive_Positive_Dev_f2 and Dev_f2>Max_Consecutive_Negative_Dev_f2:   
        arr_f2.append(dt)   #if all conditions are satisfied then it'll store that time in arr 
        i=i+1            #increment of i (i is for consecutive points)
      else:           #if all 4 conditions are not fulfilled then i will be zero and array will be empty 
        j=i
        temp_f2 = arr_f2
        i=0
        arr_f2 = []
    if j > min_no_of_continuous_value_f2:
      #print("f2")
      start_point_algo = 0
      end_point_algo = 0
      start_point_algo = temp_f2[0]
      end_point_algo = temp_f2[len(temp_f2)-1]
      
      j=0
      false_positive(start_point_algo,end_point_algo)


      

    

FALSE POSITIVE WORD DETECTED:
Word: and start_point_aws: 6.32 end_point_aws: 7.12 Duration : 0.7999999999999998, Actual_duration: 0.16999999999999993, confidence:1.0
start_point_algo 6.37326530612245, end_point_algo 6.54326530612245


FILLER WORD:
Word: and start_point_aws: 6.32 end_point_aws: 7.12Duration : 0.7999999999999998, Actual_duration: 0.21999999999999975, confidence:1.0
start_point_algo 6.8532653061224496, end_point_algo 7.073265306122449


FALSE POSITIVE WORD DETECTED:
Word: are start_point_aws: 7.13 end_point_aws: 7.37 Duration : 0.2400000000000002, Actual_duration: 0.1299999999999999, confidence:0.566
start_point_algo 7.17326530612245, end_point_algo 7.30326530612245


FALSE POSITIVE WORD DETECTED:
Word: and start_point_aws: 10.87 end_point_aws: 11.14 Duration : 0.27000000000000135, Actual_duration: 0.129999999999999, confidence:1.0
start_point_algo 10.913265306122451, end_point_algo 11.04326530612245


FALSE POSITIVE WORD DETECTED:
Word: success start_point_aws: 11.64 end